# ETL to Darwin Core

In this notebook, the mapping from the Gull bird tracking to a Darwin Core standard. 

The starting point of this data-publication process is the output of the monthly running data pipeline/ETL of the raw logs towards an processed logs version as it is provided to the researchers. Technical setup explained [here](https://github.com/inbo/bird-tracking-aws) and processing steps available as R code [here](https://github.com/inbo/bird-tracking-etl). The outcome of this ETL is a `.CSV` and a `.sqlite` data output available (for registered users) at http://birdtracking-downloader.eu-west-1.elasticbeanstalk.com/. A download from the `sqlite` version of one of the outputs can be used here to convert it to a Darwin Core.

In [2]:
import os
import gzip
import shutil

import pandas as pd

In [3]:
path = '2017-03-02-processed-logs.db.gz'

In [4]:
if os.path.isfile(path.rstrip('.gz')):
    print("sqlite file available for analysis")
elif os.path.isfile(path):
    with gzip.open(path, 'rb') as src, open(path.rstrip('.gz'), 'wb') as dest:
            shutil.copyfileobj(src, dest)
    os.remove(path)
    print("sqlite file extracted from zip-folder")
else:
    print("No sqlite or zip available, pleas provide file.")

sqlite file available for analysis


In [5]:
from sqlalchemy import create_engine

In [6]:
dbname = "sqlite:///" + path.rstrip('.gz')
engine = create_engine(dbname)

Extract Gull occurrences:

In [7]:
data = pd.read_sql_query('SELECT * FROM processed_logs as pl WHERE species_code = "lbbg" or species_code = "hg" LIMIT 20000', engine)

In [8]:
data.shape

(20000, 49)

In [9]:
data["species_code"].unique()

array(['lbbg'], dtype=object)

In [10]:
data.columns

Index(['OGC_FID', 'project_leader', 'device_info_serial', 'bird_name',
       'ring_code', 'colour_ring_code', 'species_code', 'scientific_name',
       'catch_weight', 'sex', 'catch_location', 'tracking_started_at',
       'tracking_ended_at', 'is_active', 'remarks', 'colony_latitude',
       'colony_longitude', 'date_time', 'latitude', 'longitude', 'altitude',
       'pressure', 'temperature', 'satellites_used', 'gps_fixtime',
       'positiondop', 'h_accuracy', 'v_accuracy', 'x_speed', 'y_speed',
       'z_speed', 'speed_accuracy', 'userflag', 'speed_3d', 'speed_2d',
       'direction', 'altitude_agl', 'calc_year', 'calc_month', 'calc_hour',
       'calc_time_diff', 'calc_distance_diff', 'calc_speed_2d',
       'calc_distance_to_colony', 'calc_sunlight', 'calc_outlier',
       'calc_corine_value', 'calc_corine_legend', 'the_geom'],
      dtype='object')

In [11]:
sample_outcome = pd.read_csv("dwc-occurrence-100.csv")

In [12]:
sample_outcome.head()

,occurrenceid,type,modified,language,license,rightsholder,accessrights,datasetid,institutioncode,datasetname,...,phylum,class,_order,family,genus,specificepithet,taxonrank,scientificnameauthorship,vernacularname,nomenclaturalcode
0,703:20130530182507,Event,2015-04-29T09:34:49Z,en,http://creativecommons.org/publicdomain/zero/1.0/,INBO,http://www.inbo.be/en/norms-for-data-use,http://doi.org/10.15468/02omly,INBO,Bird tracking - GPS tracking of Lesser Black-b...,...,Chordata,Aves,Charadriiformes,Laridae,Larus,fuscus,species,"Linnaeus, 1758",Lesser Black-backed Gull,ICZN
1,703:20130530185523,Event,2015-04-29T09:34:49Z,en,http://creativecommons.org/publicdomain/zero/1.0/,INBO,http://www.inbo.be/en/norms-for-data-use,http://doi.org/10.15468/02omly,INBO,Bird tracking - GPS tracking of Lesser Black-b...,...,Chordata,Aves,Charadriiformes,Laridae,Larus,fuscus,species,"Linnaeus, 1758",Lesser Black-backed Gull,ICZN
2,703:20130530192538,Event,2015-04-29T09:34:49Z,en,http://creativecommons.org/publicdomain/zero/1.0/,INBO,http://www.inbo.be/en/norms-for-data-use,http://doi.org/10.15468/02omly,INBO,Bird tracking - GPS tracking of Lesser Black-b...,...,Chordata,Aves,Charadriiformes,Laridae,Larus,fuscus,species,"Linnaeus, 1758",Lesser Black-backed Gull,ICZN
3,703:20130530195536,Event,2015-04-29T09:34:49Z,en,http://creativecommons.org/publicdomain/zero/1.0/,INBO,http://www.inbo.be/en/norms-for-data-use,http://doi.org/10.15468/02omly,INBO,Bird tracking - GPS tracking of Lesser Black-b...,...,Chordata,Aves,Charadriiformes,Laridae,Larus,fuscus,species,"Linnaeus, 1758",Lesser Black-backed Gull,ICZN
4,703:20130530202528,Event,2015-04-29T09:34:49Z,en,http://creativecommons.org/publicdomain/zero/1.0/,INBO,http://www.inbo.be/en/norms-for-data-use,http://doi.org/10.15468/02omly,INBO,Bird tracking - GPS tracking of Lesser Black-b...,...,Chordata,Aves,Charadriiformes,Laridae,Larus,fuscus,species,"Linnaeus, 1758",Lesser Black-backed Gull,ICZN


## Pythonic ETL

In [13]:
data["scientific_name"].unique()

array(['Larus fuscus'], dtype=object)

In [14]:
data["date_time"] = pd.to_datetime(data["date_time"])

__Remark__: The dates are already inn UTC, so no time zone shifting is required:

In [15]:
df_dwc = pd.DataFrame(data["device_info_serial"].astype(str) + ":" +  data["date_time"].dt.strftime("%Y%M%d%H%M%S"), columns=["occurrence_ID"])

In [16]:
df_dwc["type"] = "Event"
df_dwc["language"] = "en"
df_dwc["license"] = "http://creativecommons.org/publicdomain/zero/1.0/"
df_dwc["rightsholder"] = "INBO"
df_dwc["accessRights"] = "http://www.inbo.be/en/norms-for-data-use"
df_dwc["datasetID"] = "http://dataset.inbo.be/bird-tracking-wmh-occurrences"
df_dwc["institutionCode"] = "INBO"
df_dwc["datasetName"] = "Bird tracking - GPS tracking of Western Marsh Harriers breeding near the Belgium-Netherlands border"
df_dwc["ownerInstitutionCode"] = "INBO"
df_dwc["basisOfRecord"] = "MachineObservation"
df_dwc["informationWithheld"] = "see metadata"

**ISSUE**: to_char(t.updated_at at time zone 'UTC','YYYY-MM-DD"T"HH24:MI:SS"Z"') as modified,  # updated_at is a CARTO thing...

In [17]:
import datetime

In [18]:
df_dwc["modified"] = datetime.datetime.now().strftime("%Y-%M-%dT%H:%M:%SZ") # proposal is to use 'now', i.e. last run of the notebook...

In [19]:
df_dwc["dynamicProperties"] = "{'device_info_serial':" + data["device_info_serial"].astype(str) + "}"
df_dwc["sex"] = data["sex"]
df_dwc["lifeStage"] = "adult"
df_dwc["organismID"] = data["ring_code"]
df_dwc["organismName"] = data["bird_name"]
df_dwc["samplingProtocol"] = "doi:10.1007/s10336-012-0908-1"

In [20]:
def samplingeff(x):
    if x > 0.:
        return "{'secondsSinceLastOccurrence':" + str(round(x)) +  "}"
    else:
        return "{'secondsSinceLastOccurrence':}"

In [21]:
df_dwc["intervalinseconds"] = data["date_time"].diff(1).dt.seconds
df_dwc["samplingEffort"] = df_dwc["intervalinseconds"].apply(samplingeff)
df_dwc = df_dwc.drop("intervalinseconds", axis=1)

In [22]:
df_dwc["eventDate"] = data["date_time"].dt.strftime("%Y-%M-%dT%H:%M:%SZ")

In [23]:
df_dwc["minimumElevationInMeters"] = "0"
df_dwc["minimumDistanceAboveSurfaceInMeters"] = data["altitude"]
df_dwc["decimalLatitude"] = data["latitude"]
df_dwc["decimalLongitude"] = data["longitude"]
df_dwc["geodeticDatum"] = "WGS84"

In [24]:
df_dwc["coordinateUncertaintyInMeters"] = round(data["h_accuracy"].where(data["h_accuracy"].notnull(), 30)).astype(int)

In [25]:
df_dwc["georeferencedDate"] = data["date_time"].dt.strftime("%Y-%M-%dT%H:%M:%SZ")

In [26]:
df_dwc["georeferenceProtocol"] = 'doi:10.1080/13658810412331280211'
df_dwc["georeferenceSources"] = 'GPS'
df_dwc["georeferenceVerificationStatus"] = 'unverified'

In [27]:
df_dwc["scientificName"] = data["scientific_name"]
df_dwc["kingdom"] = 'Animalia'
df_dwc["phylum"] = 'Chordata'
df_dwc["class"] = 'Aves'
df_dwc["order"] = 'Charadriiformes'
df_dwc["family"] = 'Laridae'
df_dwc["taxonRank"] = 'species'

In [28]:
df_dwc[["genus", "specificEpithet"]] = data["scientific_name"].str.split().apply(pd.Series)

In [29]:
df_dwc["scientificNameAuthorship"] = data["species_code"].replace(to_replace=["lbbg", "hg"], value=["Linnaeus, 1758", "Pontoppidan, 1763"])
df_dwc["vernacularName"] = data["species_code"].replace(to_replace=["lbbg", "hg"], value=["Lesser Black-backed Gull", "Herring Gull"])
df_dwc["nomenclaturalCode"] = "ICZN"

In [30]:
df_dwc = df_dwc.sort_values(["occurrence_ID", "eventDate"])

In [31]:
df_dwc.head()

,occurrence_ID,type,language,license,rightsholder,accessRights,datasetID,institutionCode,datasetName,ownerInstitutionCode,...,phylum,class,order,family,taxonRank,genus,specificEpithet,scientificNameAuthorship,vernacularName,nomenclaturalCode
93,703:20130028220009,Event,en,http://creativecommons.org/publicdomain/zero/1.0/,INBO,http://www.inbo.be/en/norms-for-data-use,http://dataset.inbo.be/bird-tracking-wmh-occur...,INBO,Bird tracking - GPS tracking of Western Marsh ...,INBO,...,Chordata,Aves,Charadriiformes,Laridae,species,Larus,fuscus,"Linnaeus, 1758",Lesser Black-backed Gull,ICZN
102,703:20130129030144,Event,en,http://creativecommons.org/publicdomain/zero/1.0/,INBO,http://www.inbo.be/en/norms-for-data-use,http://dataset.inbo.be/bird-tracking-wmh-occur...,INBO,Bird tracking - GPS tracking of Western Marsh ...,INBO,...,Chordata,Aves,Charadriiformes,Laridae,species,Larus,fuscus,"Linnaeus, 1758",Lesser Black-backed Gull,ICZN
106,703:20130129040135,Event,en,http://creativecommons.org/publicdomain/zero/1.0/,INBO,http://www.inbo.be/en/norms-for-data-use,http://dataset.inbo.be/bird-tracking-wmh-occur...,INBO,Bird tracking - GPS tracking of Western Marsh ...,INBO,...,Chordata,Aves,Charadriiformes,Laridae,species,Larus,fuscus,"Linnaeus, 1758",Lesser Black-backed Gull,ICZN
48,703:20130222060208,Event,en,http://creativecommons.org/publicdomain/zero/1.0/,INBO,http://www.inbo.be/en/norms-for-data-use,http://dataset.inbo.be/bird-tracking-wmh-occur...,INBO,Bird tracking - GPS tracking of Western Marsh ...,INBO,...,Chordata,Aves,Charadriiformes,Laridae,species,Larus,fuscus,"Linnaeus, 1758",Lesser Black-backed Gull,ICZN
110,703:20130329050316,Event,en,http://creativecommons.org/publicdomain/zero/1.0/,INBO,http://www.inbo.be/en/norms-for-data-use,http://dataset.inbo.be/bird-tracking-wmh-occur...,INBO,Bird tracking - GPS tracking of Western Marsh ...,INBO,...,Chordata,Aves,Charadriiformes,Laridae,species,Larus,fuscus,"Linnaeus, 1758",Lesser Black-backed Gull,ICZN
